## Quantum Cryptography & Security: lab report 03 - QKD

## Mounting Google Drive, dataset & imports

In [ ]:
# This script would mount the drive onto the Colab Notebook
# If there is a drive mounted, the notebook would return:
# "Mounted at /content/drive" or "Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)"
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to experimental data
experimental_data_path = "/content/drive/My Drive/QCS-Lab-03-QKD-Data/"

In [ ]:
import argparse
import numpy as np
import pandas as pd
import os
import warnings
from google.colab import files as gc_files

In [ ]:
# Ignore all warnings
warnings.filterwarnings("ignore")

## Decoding the keys from data files

In [ ]:
def decoding_keys(filename, maxiter):
    '''
    Read the data from the filename file given as argument and decode the keys encoded.

    Each file contains key blocks of different lengths. A file begins with 8 bytes that code for
    a uint64 big-endian, which is the length N of the block (in bytes).
    After these first 8 bytes, N bytes (= 8*N bits) of raw keys follow.
    After the N bytes, another block begins.

    Each QKD state is represented by two bits.
    Encoding of raw-keys.alice: (00, H), (01, V), (10, D), (11, A) A not used in the three state protocol
    Encoding of raw-keys.decoy: (00, Strong Intensity), (01, Low Intensity), (10, Unused), (11, Unused)
    Encoding of raw-keys.bob: (00, H), (01, V), (10, D), (11, A).

    This code saves the decoded keys in a csv file with the corresponding block length.

    Parameters:
      filename: path to the file to read.
      maxiter: maximum number of keys to read.
    '''
    keys = []
    block_sizes = []
    # Decode file
    bytes = np.fromfile(filename, dtype='uint8')
    i = 0
    while i < len(bytes):
        block_size = int.from_bytes(bytes[i:i+8], byteorder='big')
        i += 8
        b = bytes[i:i + block_size]
        key = np.unpackbits(b, bitorder='little')
        i += block_size
        keys.append(key)
        block_sizes.append(block_size)
    # Decode keys
    decoded_states = []
    decoded_keys = []
    decoded_basis = []
    count = 0
    for k in keys:
        print('Decoding key:', count, 'of', len(keys))
        states = []
        basis = []
        bits = []
        k = np.split(k, len(k)//2)
        if filename[-5:] == 'alice' or filename[-3:] == 'bob':
            for pair in k: # 0 basis Z (key), 1 basis X (check), H/D bit 1, V/A bit 0
                if (pair==[0, 0]).all():
                    states.append('H')
                    bits.append('0')
                    basis.append('0')
                elif (pair==[0, 1]).all():
                    states.append('V')
                    bits.append('1')
                    basis.append('0')
                elif (pair==[1, 0]).all():
                    states.append('D')
                    bits.append('0')
                    basis.append('1')
                elif (pair==[1, 1]).all():
                    states.append('A')
                    bits.append('1')
                    basis.append('1')
            decoded_keys.append(''.join(bits))
            decoded_states.append(''.join(states))
            decoded_basis.append(''.join(basis))
            if maxiter != None and count == maxiter - 1:
              break
            count += 1
        else:
            for pair in k:
                if (pair==[0, 0]).all():
                    states.append('S')
                elif (pair==[0, 1]).all():
                    states.append('L')
                elif (pair==[1, 0]).all():
                    states.append('Unused')
                elif (states==[1, 1]).all():
                    states.append('Unused')
            decoded_states.append(''.join(states))
            if maxiter != None and count == maxiter - 1:
              break
            count += 1
    if filename[-5:] == 'alice' or filename[-3:] == 'bob':
        decoded_keys = np.array(decoded_keys, dtype=object)
        decoded_basis = np.array(decoded_basis, dtype=object)
    block_sizes = np.array(block_sizes, dtype=int)
    decoded_states = np.array(decoded_states, dtype=object)
    # Check if keys are correct
    print ('Decoded key (first 3 bytes):', decoded_states[0][:12])
    return decoded_keys, decoded_basis, block_sizes, decoded_states

In [ ]:
def decoding_keys_to_csv(filename, maxiter):
    if not os.path.exists('/content/decoded_keys/'):
      os.mkdir('/content/decoded_keys/')
    dir = '/content/decoded_keys/'
    decoded_keys, decoded_basis, block_sizes, decoded_states = decoding_keys(filename, maxiter)
    df = pd.DataFrame(decoded_states, columns=['states'])
    df['block_size'] = block_sizes[:maxiter]
    if filename[-5:] == 'alice' or filename[-3:] == 'bob':
        df['key'] = decoded_keys
        df['basis'] = decoded_basis
    if filename[-5:] == 'alice':
        df.to_csv(dir + '/decoded_keys_alice.csv', index=False)
    elif filename[-3:] == 'bob':
        df.to_csv(dir + '/decoded_keys_bob.csv', index=False)
    elif filename[-5:] == 'decoy':
        df.to_csv(dir + '/decoded_keys_decoy.csv', index=False)
    print("Done!")

In [ ]:
decoding_keys_to_csv(experimental_data_path + 'raw-keys.alice', 2926)
decoding_keys_to_csv(experimental_data_path + 'raw-keys.bob', 2926)
decoding_keys_to_csv(experimental_data_path + 'raw-keys.decoy', 2926)

Output streaming troncato alle ultime 5000 righe.
Decoding key: 856 of 2926
Decoding key: 857 of 2926
Decoding key: 858 of 2926
Decoding key: 859 of 2926
Decoding key: 860 of 2926
Decoding key: 861 of 2926
Decoding key: 862 of 2926
Decoding key: 863 of 2926
Decoding key: 864 of 2926
Decoding key: 865 of 2926
Decoding key: 866 of 2926
Decoding key: 867 of 2926
Decoding key: 868 of 2926
Decoding key: 869 of 2926
Decoding key: 870 of 2926
Decoding key: 871 of 2926
Decoding key: 872 of 2926
Decoding key: 873 of 2926
Decoding key: 874 of 2926
Decoding key: 875 of 2926
Decoding key: 876 of 2926
Decoding key: 877 of 2926
Decoding key: 878 of 2926
Decoding key: 879 of 2926
Decoding key: 880 of 2926
Decoding key: 881 of 2926
Decoding key: 882 of 2926
Decoding key: 883 of 2926
Decoding key: 884 of 2926
Decoding key: 885 of 2926
Decoding key: 886 of 2926
Decoding key: 887 of 2926
Decoding key: 888 of 2926
Decoding key: 889 of 2926
Decoding key: 890 of 2926
Decoding key: 891 of 2926
Decoding key: 

## Sifting procedure (basis reconciliation)

In [ ]:
def counting(df, block_length):
  raw_keys = pd.DataFrame({'time': [], 'total_pulses': [], 'm_key_mumax': [], 'm_key_mumin': [],
                           'n_key_mumax': [], 'n_key_mumin': [], 'm_check_mumax': [],
                           'm_check_mumin': [], 'n_check_mumax': [], 'n_check_mumin': []})
  i = 0
  jbit = 0
  while i < len(df):
    print('Iteration {} of {}'.format(i, len(df)))
    tot_pulses = 0
    nkmin, nkmax, ncmin, ncmax = 0, 0, 0, 0
    mkmin, mkmax, mcmin, mcmax = 0, 0, 0, 0
    key_time = 0
    while nkmax + ncmax < block_length:
      # Everytime we read a new line a second has passed
      for j in range(jbit, df['total_bits'][i]):
        tot_pulses += 1
        key_time += 1. / df['total_bits'][i] # every bit is read in a fraction of a second since the whole key is read in a second
        if not df['same_basis'][i][j]:
          continue
        if(not df['bob_basis'][i][j]) and df['decoy'][i][j] and df['correctness'][i][j]:
          nkmax += 1 # decoy strong, basis 0 (key), correct
        elif(not df['bob_basis'][i][j]) and (not df['decoy'][i][j]) and (df['correctness'][i][j]):
          nkmin += 1 # decoy weak, basis 0 (key), correct
        elif(df['bob_basis'][i][j]) and df['decoy'][i][j] and (df['correctness'][i][j]):
          ncmax += 1 # decoy strong, basis 1 (check), correct
        elif(df['bob_basis'][i][j]) and (not df['decoy'][i][j]) and (df['correctness'][i][j]):
          ncmin += 1 # decoy weak, basis 1 (check), correct
        elif(not df['bob_basis'][i][j]) and df['decoy'][i][j] and (not df['correctness'][i][j]):
          mkmax += 1 # decoy strong, basis 0 (key), incorrect
        elif(not df['bob_basis'][i][j]) and (not df['decoy'][i][j]) and (not df['correctness'][i][j]):
          mkmin += 1 # decoy weak, basis 0 (key), incorrect
        elif(df['bob_basis'][i][j]) and df['decoy'][i][j] and (not df['correctness'][i][j]):
          mcmax += 1 # decoy strong, basis 1 (check), incorrect
        elif (df['bob_basis'][i][j]) and (not df['decoy'][i][j]) and (not df['correctness'][i][j]):
          mcmin += 1 # decoy weak, basis 1 (check), incorrect
        if nkmax + ncmax >= block_length:
          # If the block is full in the middle of the reading of a key we have to restart from that point
          jbit = j
          break
      if nkmax + ncmax >= block_length:
        break
      i += 1
      jbit = 0
      if i == len(df):
        break
    raw_keys = raw_keys.append({'time': key_time, 'total_pulses': tot_pulses,
                                'm_key_mumax': mkmax, 'm_key_mumin': mkmin,
                                'n_key_mumax': nkmax, 'n_key_mumin': nkmin,
                                'm_check_mumax': mcmax, 'm_check_mumin': mcmin,
                                'n_check_mumax': ncmax, 'n_check_mumin': ncmin}, ignore_index=True)
  return raw_keys

In [ ]:
def basis_reconciliation(block_length, frac_data):
    '''
    Perfoms the basis reconciliation of the protocol, and count the number of errors and
    observations in order to generate a key of length block_length.

    Generates a csv file with the following columns:
    - time: time necessary to reconstruct the key
    - total_pulses: total number of pulses necessary to recontruct the key
    - m_key_mumax: number of errors in the key corresponding to the maximum intensity decoy
    - m_key_mumin: number of errors in the key corresponding to the minimum intensity decoy
    - n_key_mumax: number of observations in the key corresponding to the maximum intensity decoy
    - n_key_mumin: number of observations in the key corresponding to the minimum intensity decoy
    - m_check_mumax: number of errors in the check basis corresponding to the maximum intensity decoy
    - m_check_mumin: number of errors in the check basis corresponding to the minimum intensity decoy
    - n_check_mumax: number of observations in the check basis corresponding to the maximum intensity decoy
    - n_check_mumin: number of observations in the check basis corresponding to the minimum intensity decoy

    Parameters:
      block_length: number of total observations in the key basis
      frac_data: fraction of data to analyze
    '''
    a = pd.read_csv('/content/decoded_keys/decoded_keys_alice.csv')
    b = pd.read_csv('/content/decoded_keys/decoded_keys_bob.csv')
    d = pd.read_csv('/content/decoded_keys/decoded_keys_decoy.csv')
    n_data = int(len(a) * frac_data)
    a = a[:n_data]
    b = b[:n_data]
    d = d[:n_data]
    print('Processing data...')
    a['key'] = a['key'].apply(lambda x: np.array(list(x), dtype=int)) # true if it is the basis X (not key)
    a['basis'] = a['basis'].apply(lambda x: np.array(list(x), dtype=int))
    b['key'] = b['key'].apply(lambda x: np.array(list(x), dtype=int))
    b['basis'] = b['basis'].apply(lambda x: np.array(list(x), dtype=int))
    d = d.drop(columns=['block_size'])
    b = b.drop(columns=['block_size'])
    df = pd.concat([a, b, d], axis=1)
    df.columns = ['alice_state','total_bits', 'alice_key', 'alice_basis',
                  'bob_state', 'bob_key', 'bob_basis', 'decoy_state']
    print('Checking basis and correctness...')
    df['total_bits'] = df.apply(lambda x: len(x['alice_key']), axis=1)
    df['same_basis'] = df.apply(lambda x: x['alice_basis'] == x['bob_basis'], axis=1)
    df['same_bits'] = df.apply(lambda x: x['alice_key'] == x['bob_key'], axis=1)
    df['correctness'] = df.apply(lambda x: x['same_basis'] & x['same_bits'], axis=1)
    # True if the decoy is the strong one
    df['decoy'] = df.apply(lambda x: list(x['decoy_state']) == np.full(len(x['decoy_state']), 'S'), axis=1)
    df['time'] = np.arange(len(df))
    df.to_csv('/content/decoded_keys/basis_reconciliation.csv', index=False)
    # Basis reconciliation:
    # Alice and Bob announce their basis and intensity choices over an authenticated public channel
    # and identify the following sets: X_{k} := {i: a_{i}=b_{i}=X and k_{i}=k} and Z_{k} := {i: a_{i}=b_{i}=Z and k_{i}=k}
    results = counting(df, block_length=block_length)
    # Convert the results into a csv file
    if not os.path.exists('/content/results/'):
      os.mkdir('/content/results/')
    results.to_csv('/content/results/countings_{}_{}.csv'.format(block_length, frac_data), index=False)

In [ ]:
basis_reconciliation(100000, 1)

Processing data...
Checking basis and correctness...
Iteration 0 of 2926
Iteration 9 of 2926
Iteration 18 of 2926
Iteration 27 of 2926
Iteration 37 of 2926
Iteration 46 of 2926
Iteration 55 of 2926
Iteration 65 of 2926
Iteration 74 of 2926
Iteration 84 of 2926
Iteration 93 of 2926
Iteration 102 of 2926
Iteration 112 of 2926
Iteration 121 of 2926
Iteration 130 of 2926
Iteration 140 of 2926
Iteration 149 of 2926
Iteration 158 of 2926
Iteration 168 of 2926
Iteration 177 of 2926
Iteration 186 of 2926
Iteration 195 of 2926
Iteration 205 of 2926
Iteration 214 of 2926
Iteration 223 of 2926
Iteration 233 of 2926
Iteration 242 of 2926
Iteration 251 of 2926
Iteration 261 of 2926
Iteration 270 of 2926
Iteration 280 of 2926
Iteration 289 of 2926
Iteration 298 of 2926
Iteration 308 of 2926
Iteration 317 of 2926
Iteration 326 of 2926
Iteration 336 of 2926
Iteration 345 of 2926
Iteration 355 of 2926
Iteration 364 of 2926
Iteration 373 of 2926
Iteration 383 of 2926
Iteration 392 of 2926
Iteration 401 

In [ ]:
basis_reconciliation(1000000, 1)

Processing data...
Checking basis and correctness...
Iteration 0 of 2926
Iteration 93 of 2926
Iteration 186 of 2926
Iteration 280 of 2926
Iteration 373 of 2926
Iteration 467 of 2926
Iteration 559 of 2926
Iteration 652 of 2926
Iteration 746 of 2926
Iteration 839 of 2926
Iteration 931 of 2926
Iteration 1025 of 2926
Iteration 1118 of 2926
Iteration 1211 of 2926
Iteration 1304 of 2926
Iteration 1397 of 2926
Iteration 1490 of 2926
Iteration 1583 of 2926
Iteration 1676 of 2926
Iteration 1769 of 2926
Iteration 1862 of 2926
Iteration 1956 of 2926
Iteration 2049 of 2926
Iteration 2142 of 2926
Iteration 2236 of 2926
Iteration 2330 of 2926
Iteration 2423 of 2926
Iteration 2517 of 2926
Iteration 2610 of 2926
Iteration 2703 of 2926
Iteration 2797 of 2926
Iteration 2890 of 2926


In [ ]:
basis_reconciliation(10000000, 1)

Processing data...
Checking basis and correctness...
Iteration 0 of 2926
Iteration 931 of 2926
Iteration 1862 of 2926
Iteration 2797 of 2926


## Security parameters

In [ ]:
class parameters():
  '''
  Parameters:
    alice_key_basis_prob: probability of Alice using the key basis
    alice_check_basis_prob: probability of Alice using the check basis
    bob_key_basis_prob: probability of Bob using the key basis
    bob_check_basis_prob: probability of Bob using the check basis
    decoy_strong_prob: probability of the decoy being the strong one
    decoy_weak_prob: probability of the decoy being the weak one
    decoy_strong_intensity: intensity of the strong decoy
    decoy_weak_intensity: intensity of the weak decoy
    secrecy: secrecy of the protocol
    correctness: correctness of the protocol
    lambda_EC: Alice and Bob perform an error-correction step that reveals at most lambda_{EC} bits of information
  '''
  def __init__(self, alice_key_basis_prob, alice_check_basis_prob,
               bob_key_basis_prob, bob_check_basis_prob,
               decoy_strong_prob, decoy_weak_prob,
               decoy_strong_intensity, decoy_weak_intensity,
               secrecy=1e-9, correctness=1e-15,
               lambda_EC=1.16):
    if alice_check_basis_prob + alice_key_basis_prob != 1:
      raise ValueError('Aice basis probability must sum to 1!')
    if alice_check_basis_prob > 1 or alice_check_basis_prob < 0 or alice_key_basis_prob > 1 or alice_key_basis_prob < 0:
      raise ValueError('Aice basis probability must be between 0 and 1!')
    if bob_check_basis_prob + bob_key_basis_prob != 1:
      raise ValueError('Bob basis probability must sum to 1!')
    if bob_check_basis_prob > 1 or bob_check_basis_prob < 0 or bob_key_basis_prob > 1 or bob_key_basis_prob < 0:
      raise ValueError('Bob basis probability must be between 0 and 1!')
    if decoy_strong_prob + decoy_weak_prob != 1:
      raise ValueError('Decoy probability must sum to 1!')
    if decoy_strong_prob > 1 or decoy_strong_prob < 0 or decoy_weak_prob > 1 or decoy_weak_prob < 0:
      raise ValueError('Decoy probability must be between 0 and 1!')
    if decoy_strong_intensity < decoy_weak_intensity:
      warnings.warn('Decoy strong intensity must be greater than decoy weak intensity!')
      decoy_strong_intensity, decoy_weak_intensity = decoy_weak_intensity, decoy_strong_intensity
    if lambda_EC < 0:
      raise ValueError('lambda_{EC} must be positive, but it is {}'.format(lambda_EC))
    self.a_key_basis_prob = alice_key_basis_prob
    self.a_check_basis_prob = alice_check_basis_prob
    self.b_key_basis_prob = bob_key_basis_prob
    self.b_check_basis_prob = bob_check_basis_prob
    self.d_strong_prob = decoy_strong_prob
    self.d_weak_prob = decoy_weak_prob
    self.d_strong_intensity = decoy_strong_intensity
    self.d_weak_intensity = decoy_weak_intensity
    self.secrecy = secrecy
    self.correctness = correctness
    self.lambda_EC = lambda_EC
    # tau_{n} = sumk p_{k} e^{−k} k^{n}/n! is the total probability to send an n photon state
    self.tau_0 = self.d_weak_prob * np.exp(-self.d_weak_intensity) + self.d_strong_prob * np.exp(-self.d_strong_intensity)
    self.tau_1 = self.d_weak_prob * np.exp(-self.d_weak_intensity) * self.d_weak_intensity + self.d_strong_prob * np.exp(-self.d_strong_intensity) * self.d_strong_intensity
    # Fixed parameters
    self.a = 6
    self.b = 19

## Helper functions

In [ ]:
def gamma(a, b, c, d, e_b, e_c, e_d): # not considering error on a
  log = np.log(((c + d) / (c * d * b * (1 - b))) * (21 / a)**2)
  gamma_p = np.sqrt(((c + d) * (1 - b) * b) / (c * d) * log)
  gamma = gamma_p / np.log(2)
  partial_b = ((c + d) * (2 * b - 1) * (log - 1)) / (2 * c * d * np.log(2) * gamma_p)
  partial_c = (b * (b - 1) * (log + 1)) / (2 * c * c * np.log(2) * gamma_p)
  partial_d = (b * (b - 1) * (log + 1)) / (2 * d * d * np.log(2) * gamma_p)
  error = np.sqrt(e_b**2 * partial_b**2 + e_c**2 * partial_c**2 + e_d**2 * partial_d**2)
  return gamma, error

def bin_entropy(x, e_x):
  entropy = -x * np.log2(x) - (1 - x) * np.log2(1 - x)
  error = (np.log(1 - x) - np.log(x)) * e_x / np.log(2)
  return entropy, error

## Security bounds

In [ ]:
def security_bounds(block_size, frac_data, time_evolution,
                    alice_key_basis_prob, alice_check_basis_prob,
                    bob_key_basis_prob, bob_check_basis_prob,
                    decoy_strong_prob, decoy_weak_prob,
                    decoy_strong_intensity, decoy_weak_intensity):
    '''
    Computes QBER and SKR with the respective error as proposed in:

    [1] D. Rusca, A. Boaron, F. Grunenfelder, A. Martin, and H. Zbinden.
    Finite-key analysis for the 1-decoy state QKD protocol.
    Applied Physics Letters, 112(17):171104, April 2018.
    DOI: 10.1063/1.5023340.
    URL: https://doi.org/10.1063/1.5023340;

    [2] C. C. W. Lim, M. Curty, N. Walenta, F. Xu, and H. Zbinden.
    Concise security bounds for practical decoy-state quantum key distribution.
    Phys. Rev. A, 89:022307, February 2014.
    DOI: 10.1103/PhysRevA.89.022307.
    URL: https://link.aps.org/doi/10.1103/PhysRevA.89.022307.

    Parameters:
      block_size: block size
      frac_data: fraction of data to read
      alice_key_basis_prob: probability of Alice using the key basis
      alice_check_basis_prob: probability of Alice using the check basis
      bob_key_basis_prob: probability of Bob using the key basis
      bob_check_basis_prob: probability of Bob using the check basis
      decoy_strong_prob: probability of the decoy being the strong one
      decoy_weak_prob: probability of the decoy being the weak one
      decoy_strong_intensity: intensity of the strong decoy
      decoy_weak_intensity: intensity of the weak decoy
    '''

    p = parameters(alice_key_basis_prob, alice_check_basis_prob,
                   bob_key_basis_prob, bob_check_basis_prob,
                   decoy_strong_prob, decoy_weak_prob,
                   decoy_strong_intensity, decoy_weak_intensity,
                   secrecy=1e-9, correctness=1e-15,
                   lambda_EC=1.16)

    if not time_evolution:
        df = pd.read_csv('/content/results/countings_{}_{}.csv'.format(block_size, frac_data))
        df = df.drop(df.index[-1], axis=0)
    else:
        df = pd.read_csv('/content/results/time_evolution_{}_{}.csv'.format(block_size, frac_data))

    # Errors on countings and finite key effect
    for c in df.columns[1:]:
        df[c + '_err'] = np.sqrt(df[c])

    df['n_key_tot'] = df['n_key_mumax'] + df['n_key_mumin']
    df['n_key_tot_err'] = np.sqrt(df['n_key_mumax_err']**2 + df['n_key_mumin_err']**2)

    df['n_check_tot'] = df['n_check_mumax'] + df['n_check_mumin']
    df['n_check_tot_err'] = np.sqrt(df['n_check_mumax_err']**2 + df['n_check_mumin_err']**2)

    df['m_key_tot'] = df['m_key_mumax'] + df['m_key_mumin']
    df['m_key_tot_err'] = np.sqrt(df['m_key_mumax_err']**2 + df['m_key_mumin_err']**2)

    df['m_check_tot'] = df['m_check_mumax'] + df['m_check_mumin']
    df['m_check_tot_err'] = np.sqrt(df['m_check_mumax_err']**2 + df['m_check_mumin_err']**2)

    deltaN_key = np.sqrt(0.5 * df['n_key_tot'] * np.log(19/p.secrecy))
    deltaM_key = np.sqrt(0.5 * df['m_key_tot'] * np.log(19/p.secrecy))
    deltaN_check = np.sqrt(0.5 * df['n_check_tot'] * np.log(19/p.secrecy))
    deltaM_check = np.sqrt(0.5 * df['m_check_tot'] * np.log(19/p.secrecy))

    strong_prefactor = np.exp(p.d_strong_intensity) / p.d_strong_prob
    weak_prefactor = np.exp(p.d_weak_intensity) / p.d_weak_prob

    df['n_key_mumax_plus'] = strong_prefactor * (df['n_key_mumax'] + deltaN_key)
    df['n_key_mumax_min'] = strong_prefactor * (df['n_key_mumax'] - deltaN_key)
    df['n_key_mumax_err_hoeff'] = strong_prefactor * np.sqrt(df['n_key_mumax_err']**2 + (np.log(19./p.secrecy) / (8 * df['n_key_tot'])) * df['n_key_tot_err']**2)

    df['n_key_mumin_plus'] = weak_prefactor * (df['n_key_mumin'] + deltaN_key)
    df['n_key_mumin_min'] = weak_prefactor * (df['n_key_mumin'] - deltaN_key)
    df['n_key_mumin_err_hoeff'] = weak_prefactor * np.sqrt(df['n_key_mumin_err']**2 + (np.log(19./p.secrecy) / (8 * df['n_key_tot'])) * df['n_key_tot_err']**2)

    df['n_check_mumax_plus'] = strong_prefactor * (df['n_check_mumax'] + deltaN_check)
    df['n_check_mumax_min'] = strong_prefactor * (df['n_check_mumax'] - deltaN_check)
    df['n_check_mumax_err_hoeff'] = strong_prefactor * np.sqrt(df['n_check_mumax_err']**2 + (np.log(19./p.secrecy) / (8 * df['n_check_tot'])) * df['n_check_tot_err']**2)

    df['n_check_mumin_plus'] = weak_prefactor * (df['n_check_mumin'] + deltaN_check)
    df['n_check_mumin_min'] = weak_prefactor * (df['n_check_mumin'] - deltaN_check)
    df['n_check_mumin_err_hoeff'] = weak_prefactor * np.sqrt(df['n_check_mumin_err']**2 + (np.log(19./p.secrecy) / (8 * df['n_check_tot'])) * df['n_check_tot_err']**2)

    df['m_key_mumax_plus'] = strong_prefactor * (df['m_key_mumax'] + deltaM_key)
    df['m_key_mumax_min'] = strong_prefactor * (df['m_key_mumax'] - deltaM_key)
    df['m_key_mumax_err_hoeff'] = strong_prefactor * np.sqrt(df['m_key_mumax_err']**2 + (np.log(19./p.secrecy) / (8 * df['m_key_tot'])) * df['m_key_tot_err']**2)

    df['m_key_mumin_plus'] = weak_prefactor * (df['m_key_mumin'] + deltaM_key)
    df['m_key_mumin_min'] = weak_prefactor * (df['m_key_mumin'] - deltaM_key)
    df['m_key_mumin_err_hoeff'] = strong_prefactor * np.sqrt(df['m_key_mumin_err']**2 + (np.log(19./p.secrecy) / (8 * df['m_key_tot'])) * df['m_key_tot_err']**2)

    df['m_check_mumax_plus'] = strong_prefactor * (df['m_check_mumax'] + deltaM_check)
    df['m_check_mumax_min'] = strong_prefactor * (df['m_check_mumax'] - deltaM_check)
    df['m_check_mumax_err_hoeff'] = weak_prefactor * np.sqrt(df['m_check_mumax_err']**2 + (np.log(19./p.secrecy) / (8 * df['m_check_tot'])) * df['m_check_tot_err']**2)

    df['m_check_mumin_plus'] = weak_prefactor * (df['m_check_mumin'] + deltaM_check)
    df['m_check_mumin_min'] = weak_prefactor * (df['m_check_mumin'] - deltaM_check)
    df['m_check_mumin_err_hoeff'] = weak_prefactor * np.sqrt(df['m_check_mumin_err']**2 + (np.log(19./p.secrecy) / (8 * df['m_check_tot'])) * df['m_check_tot_err']**2)

    # QBER
    df['QBER_key'] = df['m_key_tot'] / df['n_key_tot']
    df['QBER_key_err'] = np.sqrt(df['m_key_tot_err']**2 / df['n_key_tot']**2 + df['m_key_tot']**2 * df['n_key_tot_err']**2 / df['n_key_tot']**4)

    df['QBER_check'] = df['m_check_tot'] / df['n_check_tot']
    df['QBER_check_err'] = np.sqrt(df['m_check_tot_err']**2 / df['n_check_tot']**2 + df['m_check_tot']**2 * df['n_check_tot_err']**2 / df['n_check_tot']**4)

    # Lower bound of zero photon detections
    df['s_key_0_low'] = (p.tau_0 / (p.d_strong_intensity - p.d_weak_intensity)) * (p.d_strong_intensity * df['n_key_mumin_min'] - p.d_weak_intensity * df['n_key_mumax_plus'])
    df['s_key_0_low_err'] = (p.tau_0 / (p.d_strong_intensity - p.d_weak_intensity)) * np.sqrt(p.d_strong_intensity**2 * (df['n_key_mumin_err_hoeff']**2 +
                             p.d_weak_intensity**2 * df['n_key_mumax_err_hoeff']**2))

    # Upper bound of zero photon detections
    df['s_key_0_up_strong'] = 2 * (p.tau_0 * df['m_key_mumax_plus'] + np.sqrt(0.5 * df['n_key_tot'] * np.log(19/p.secrecy)))
    df['s_key_0_up_strong_err'] = 2 * np.sqrt((p.tau_0 * df['m_key_mumax_err_hoeff'])**2 + (np.log(19/p.secrecy) / (8 * df['n_key_tot'])) * df['n_key_tot_err']**2)

    # Lower bound of one photon detections key
    df['s_key_1_low'] = (p.tau_1 * p.d_strong_intensity / (p.d_weak_intensity * (p.d_strong_intensity - p.d_weak_intensity))) * (df['n_key_mumin_min']
                        - (p.d_weak_intensity / p.d_strong_intensity)**2 * df['n_key_mumax_plus']
                        - (p.d_strong_intensity**2 - p.d_weak_intensity**2) / (p.d_strong_intensity**2 * p.tau_0) * df['s_key_0_up_strong'])

    df['s_key_1_low_err'] = (p.tau_1 * p.d_strong_intensity / (p.d_weak_intensity * (p.d_strong_intensity - p.d_weak_intensity))) * np.sqrt((df['n_key_mumin_err_hoeff']**2
                            + (p.d_weak_intensity / p.d_strong_intensity)**4 * df['n_key_mumax_err_hoeff']**2
                            + ((p.d_strong_intensity**2 - p.d_weak_intensity**2) / (p.d_strong_intensity**2))**2 * (df['s_key_0_up_strong_err'] / p.tau_0)**2))

    # Lower bound of one photon detections check (needed for phase error)
    df['s_check_0_up_weak'] = 2 * (p.tau_0 * df['m_check_mumin_plus'] + np.sqrt(0.5 * df['n_check_tot'] * np.log(19/p.secrecy)))
    df['s_check_0_up_weak_err'] = 2 * np.sqrt((p.tau_0 * df['m_check_mumax_err_hoeff'])**2 + (np.log(19/p.secrecy) / (8 * df['n_check_tot'])) * df['n_check_tot_err']**2)

    df['s_check_1_low'] = (p.tau_1 * p.d_strong_intensity / (p.d_weak_intensity * (p.d_strong_intensity - p.d_weak_intensity))) * (df['n_check_mumin_min']
                        - (p.d_weak_intensity / p.d_strong_intensity)**2 * df['n_check_mumax_plus']\
                        - (p.d_strong_intensity**2 - p.d_weak_intensity**2) / (p.d_strong_intensity**2) * (df['s_check_0_up_weak'] / p.tau_0))

    df['s_check_1_low_err'] = (p.tau_1 * p.d_strong_intensity / (p.d_weak_intensity * (p.d_strong_intensity - p.d_weak_intensity))) * np.sqrt((df['n_check_mumin_err_hoeff']**2
                            + (p.d_weak_intensity / p.d_strong_intensity)**4 * df['n_check_mumax_err_hoeff']**2
                            + ((p.d_strong_intensity**2 - p.d_weak_intensity**2) / (p.d_strong_intensity**2 * p.tau_0))**2 * df['s_check_0_up_weak_err']**2))

    df['v_check_1_up'] = (p.tau_1 / (p.d_strong_intensity - p.d_weak_intensity)) * (df['m_check_mumax_plus'] - df['m_check_mumin_min'])
    df['v_check_1_up_err'] = (p.tau_1 / (p.d_strong_intensity - p.d_weak_intensity)) * np.sqrt(df['m_check_mumax_err_hoeff']**2 + df['m_check_mumin_err_hoeff']**2)

    err_temp = np.sqrt((df['v_check_1_up_err'] / df['s_check_1_low'])**2 + (df['v_check_1_up'] * df['s_check_1_low_err'] / df['s_check_1_low']**2)**2)
    df['gamma'], df['gamma_err'] = gamma(p.secrecy, df['v_check_1_up'] / df['s_check_1_low'], df['s_key_1_low'],
                                         df['s_check_1_low'], err_temp, df['s_key_1_low_err'], df['s_check_1_low_err'])
    # Phase error
    df['phi_up'] = df['v_check_1_up'] / df['s_check_1_low'] + df['gamma']
    df['phi_up_err'] = np.sqrt(err_temp**2 + df['gamma_err']**2)

    df[df < 0] = 0 # set to zero non physical values

    entropy, error = bin_entropy(df['phi_up'], df['phi_up_err'])
    p.lambda_EC, _ = bin_entropy(np.mean(df['QBER_key']), 0)
    p.lambda_EC = p.lambda_EC * 1.16
    df['secret_key_length'] = df['s_key_0_low'] + df['s_key_1_low']*(1 - entropy) - p.lambda_EC - 6 * np.log2(19/p.secrecy) - np.log2(2/p.correctness)
    df['secret_key_length_err'] = np.sqrt(df['s_key_0_low_err']**2 + (df['s_key_1_low_err']*(1 - entropy))**2 + (df['s_key_1_low_err'] * error)**2)

    repetition_rate = 1
    df['SKR'] = df['secret_key_length'] * repetition_rate / df['total_pulses']
    df['SKR_err'] = np.sqrt((df['secret_key_length_err'] * repetition_rate / df['total_pulses'])**2 +
                            (df['secret_key_length'] * repetition_rate * df['total_pulses_err'] / df['total_pulses']**2)**2)

    if not time_evolution:
        df.to_csv('/content/results/{}_{}_results.csv'.format(block_size, frac_data), index=False)
    else:
        df.to_csv('/content/results/time_evolution_{}_{}_results.csv'.format(block_size, frac_data), index=False)

In [ ]:
security_bounds(100000, 1, None, 0.90, 0.10, 0.50, 0.50, 0.70, 0.30, 0.6, 0.1818)
security_bounds(1000000, 1, None, 0.90, 0.10, 0.50, 0.50, 0.70, 0.30, 0.6, 0.1818)
security_bounds(10000000, 1, None, 0.90, 0.10, 0.50, 0.50, 0.70, 0.30, 0.6, 0.1818)

## Time evolution

In [ ]:
def time_evolution(results_path, filename):
    '''
    From previous computations, computes the SKR and all the bounds by
    considering each time all data from the beginning to a certain time.
    This allows to show the dependence of a certain parameter on the time.
    '''
    df = pd.read_csv(filename)
    results = pd.DataFrame({'time': [], 'total_pulses': [], 'm_key_mumax': [], 'm_key_mumin': [],
                            'n_key_mumax': [], 'n_key_mumin': [], 'm_check_mumax': [],
                            'm_check_mumin': [], 'n_check_mumax': [], 'n_check_mumin': []})

    for i in range(len(df)):
      a = df[:i+1]
      a = a.sum()
      results = results.append({'time': a['time'], 'total_pulses': a['total_pulses'],
                                'm_key_mumax': a['m_key_mumax'], 'm_key_mumin': a['m_key_mumin'],
                                'n_key_mumax': a['n_key_mumax'], 'n_key_mumin': a['n_key_mumin'],
                                'm_check_mumax': a['m_check_mumax'], 'm_check_mumin': a['m_check_mumin'],
                                'n_check_mumax': a['n_check_mumax'], 'n_check_mumin': a['n_check_mumin']}, ignore_index=True)

    results.to_csv(results_path, index=False)

In [ ]:
time_evolution('/content/results/time_evolution_100000_1.csv', '/content/results/100000_1_results.csv')
time_evolution('/content/results/time_evolution_1000000_1.csv', '/content/results/1000000_1_results.csv')
time_evolution('/content/results/time_evolution_10000000_1.csv', '/content/results/10000000_1_results.csv')

## Download the results

In [ ]:
# Specify the folder path and zip filename
folder_path = "/content/decoded_keys"
zip_filename = "decoded_keys_results.zip"
# Zip all files inside the folder
!zip -r {zip_filename} {folder_path}/*
# Download the zip file
gc_files.download(zip_filename)

  adding: content/decoded_keys/basis_reconciliation.csv (deflated 78%)
  adding: content/decoded_keys/decoded_keys_alice.csv (deflated 83%)
  adding: content/decoded_keys/decoded_keys_bob.csv (deflated 77%)
  adding: content/decoded_keys/decoded_keys_decoy.csv (deflated 90%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Specify the folder path and zip filename
folder_path = "/content/results"
zip_filename = "security_analysis_results.zip"
# Zip all files inside the folder
!zip -r {zip_filename} {folder_path}/*
# Download the zip file
gc_files.download(zip_filename)

  adding: content/results/10000000_1_results.csv (deflated 53%)
  adding: content/results/1000000_1_results.csv (deflated 50%)
  adding: content/results/100000_1_results.csv (deflated 55%)
  adding: content/results/countings_10000000_1.csv (deflated 49%)
  adding: content/results/countings_1000000_1.csv (deflated 60%)
  adding: content/results/countings_100000_1.csv (deflated 67%)
  adding: content/results/time_evolution_10000000_1.csv (deflated 47%)
  adding: content/results/time_evolution_1000000_1.csv (deflated 53%)
  adding: content/results/time_evolution_100000_1.csv (deflated 56%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>